In [1]:
# Импорт необходимых модулей
from audio.audio_analyzer import AudioAnalyzer
from tracking.face_reid import FaceReId
from utils.scene_tools import (
    enrich_scenes_with_characters,
    enrich_scenes_with_audio,
    group_semantic_scenes,
    clean_and_merge_short_scenes,
    print_scenes_formatted,
)
from utils.video_tools import save_scenes_ffmpeg
from utils.audio_tools import extract_audio_ffmpeg
from sentence_transformers import SentenceTransformer
from video.video_analyzer import VideoAnalyzer

/home/artniz/hackatons/kion/kionvenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Анализ видео и выделение сцен

### В этой ячейке мы создаём объект VideoAnalyzer и анализируем видео для получения сцены и лиц, привязанных к трекам.


In [2]:
video_path = "test.mp4"

video_analyzer = VideoAnalyzer()

# Анализируем видео — получаем сцены и лица (срезы лиц из кадров)
scenes, track_faces = video_analyzer.analyze_video(video_path)

Analyzing scenes: 100%|██████████| 43/43 [00:13<00:00,  3.10it/s]


# Идентификация персонажей (Face Re-Identification)

### Здесь используем FaceReId для определения, каким персонажам принадлежат найденные лица.


In [3]:
face_reid = FaceReId()

# Получаем отображение track_id → персонаж (имя или ID)
track_id_to_person = face_reid.analyze_persons(track_faces)


Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'sdpa_kernel': '0', 'use_tf32': '1', 'fuse_conv_bias': '0', 'prefer_nhwc': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_enable': '0', 'use_ep_level_unified_stream': '0', 'device_id': '0', 'has_user_compute_stream': '0', 'gpu_external_empty_cache': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_external_free': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'enable_cuda_graph': '0', 'user_compute_stream': '0', 'cudnn_conv_use_max_workspace': '1'}}
find model: /home/artniz/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'],

Analyzing faces: 100%|██████████| 19/19 [00:15<00:00,  1.25it/s]


# Обогащение сцен информацией о персонажах

### Связываем сцены с именами персонажей на основе сопоставления track_id → персонаж.


In [4]:
scene_data = enrich_scenes_with_characters(scenes, track_id_to_person)


# Извлечение аудио из видео

### С помощью ffmpeg извлекаем аудио в формате WAV для последующего анализа.


In [5]:
audio_path = extract_audio_ffmpeg(video_path)


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

# Анализ аудио: распознавание речи и активность

### Используем AudioAnalyzer для транскрибирования речи и измерения активности звука (энергии).

In [6]:
audio_analyzer = AudioAnalyzer(audio_path)

# Получаем список сегментов с распознанным текстом
segments = audio_analyzer.transcribe_audio()

# Получаем значения энергии аудио для каждого временного фрейма
energy = audio_analyzer.detect_audio_activity(frame_duration=1.0)

# Обогащение сцен транскриптами и аудио энергией

### Добавляем к сценам распознанный текст и усреднённые показатели звуковой активности.


In [7]:
scene_data_copy = scene_data.copy()

scenes_enrich = enrich_scenes_with_audio(scene_data_copy, segments, energy)


# Семантическая группировка и очистка сцен

### Группируем сцены по сходству персонажей, текстов и аудио, затем объединяем слишком короткие сцены.


In [8]:
scenes_enrich_copy = scenes_enrich.copy()

# Загружаем модель для вычисления семантических эмбеддингов текста
sentenceTransformer = SentenceTransformer('all-MiniLM-L6-v2', device="cuda")

# Группируем похожие сцены
scenes_grouped = group_semantic_scenes(scenes_enrich_copy, sentenceTransformer)

# Убираем слишком короткие сцены, объединяя их с соседними
scenes_grouped = clean_and_merge_short_scenes(scenes_grouped, min_duration=2.0, min_words=3)

# Вывод результата

### Печатаем красиво отформатированный отчёт по сценам.


In [9]:
print_scenes_formatted(scenes_grouped)



🎬 Сцена 1
⏱ Время: 0.00 – 32.49 сек
🧍 Персонажи (characters): person_0, person_1
🔊 Средняя громкость: 0.0523
🗣 Реплика:
"Рванули в России за славой, вормию нас, кто угодно, немцы и таляны даже шавой цары, 600 тысяч тыков. Истет, что за дерьмо, Миссия на полион, ты нам дороги на карте рисовали, дорог не хиранет, наибалполучается. До смоленской остелась, немнёжка. Бонья сказяли, что это приключение на 20 минут. Скоро домой с победой. Вы"
------------------------------------------------------------

🎬 Сцена 2
⏱ Время: 32.49 – 55.98 сек
🧍 Персонажи (characters): person_0, person_1
🔊 Средняя громкость: 0.0535
🗣 Реплика:
"как гляньте, что это творите? Моленск но это пистет. Рвуски нас чуть не вайпнули. Еще император чуть не поджарился в шатре. Это просто вилы, Рвуски исполели все свои избючки. Все вебещали нам богатые Рвуски деревня, но мы жирем землю пеплом. пеплом. пеплом. Вот Сия."
------------------------------------------------------------

🎬 Сцена 3
⏱ Время: 55.98 – 63.48 сек
🧍 Персон

# Сохранение выделенных сцен в отдельные видеофайлы

### Используем ffmpeg для вырезки и сохранения сцен в отдельные файлы.


In [10]:
output_dir = "output_scenes"

save_scenes_ffmpeg(video_path, scenes_grouped, output_dir)


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab